In [1]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
import matplotlib.pyplot as plt
import glob

##Verilerin Ön İşlemesi

In [2]:
table_data = pd.read_csv("/content/drive/MyDrive/house_table.txt", header=None, sep=" ")

In [3]:
table_data = pd.read_csv("/content/drive/MyDrive/house_table.txt", header=None, sep=" ")

price = table_data.iloc[:,-1]
info_house = table_data.iloc[:,:-1]

table_x_train, table_x_test, table_y_train, table_y_test = train_test_split(info_house,price, test_size=0.33)

table_x_train = table_x_train.values
table_x_test = table_x_test.values

#x_train ve x_test için sütünlara göre bölme işlemi
room_train, floors_train, m2_train, loc_train = np.transpose(table_x_train) 
room_test, floors_test, m2_test, loc_test = np.transpose(table_x_test)
table_data

,0,1,2,3,4
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226
...,...,...,...,...,...
530,5,2.0,2066,94531,399900
531,4,3.5,9536,94531,460000
532,3,2.0,2014,94531,407000
533,4,3.0,2312,94531,419000


In [4]:
table_x_train.shape

(358, 4)

In [ ]:
price_train = np.array(table_y_train)
price_test = np.array(table_y_test)

In [ ]:
price_train

In [ ]:
x_train = {"input_room": room_train,"input_floors": floors_train,"input_m2":m2_train, "input_loc":loc_train }
x_test = {"input_room": room_test,"input_floors": floors_test,"input_m2":m2_test, "input_loc":loc_test }

##Model

In [ ]:
input_shape  = (1,)
input_room   = Input(shape=input_shape, name="input_room")
input_floors = Input(shape=input_shape, name="input_floors")
input_m2     = Input(shape=input_shape, name="input_m2")
input_loc    = Input(shape=input_shape, name="input_loc")

In [ ]:
input_layer=[input_loc,input_room,input_floors,input_m2]
input_linear_list= [input_room,input_floors,input_m2]
input_linear_list2 = Concatenate(axis=1)(input_linear_list)

l1_1 =  Dense(36,activation="relu")(input_linear_list2)

l1_2 =  Dense(9,activation="relu")(l1_1)

l1_3 =  Dense(3,activation="relu")(l1_2)

l2_1 =  Dense(9,activation="relu")(input_loc)

l2_2 =  Dense(9,activation="relu")(l2_1)

l2_3 =  Dense(1,activation="relu")(l2_2)


con = Concatenate(axis=1)([l1_3,l2_3])
output_layer = Dense(1, activation="linear")(con)
model = Model(input_layer, output_layer)



##PLOT

In [ ]:
tensorflow.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True
)

In [ ]:
model.compile(optimizer="adam",
              loss="mean_absolute_error",
              metrics="mean_absolute_error")

In [ ]:
model.fit(x_train,price_train,
          batch_size=4,
          epochs = 100,
          validation_data = (x_test,price_test)
          )